# LAB 4 HLS_PYNQ tutorial

In [ ]:
from pynq import Overlay
from pynq import allocate
import numpy as np

In [ ]:
def init_hw(filepath):
    global ol, ip
    ol = Overlay(filepath)
    ip = ol.matMul_0

#### Program board with bitstream file

In [ ]:
init_hw("/home/xilinx/matmul.bit")

#### Allocate matrix on pynq buffer

In [ ]:
DIM = 256

a = allocate(shape=((DIM, DIM)), dtype=np.int32, cacheable=True)
b = allocate(shape=((DIM, DIM)), dtype=np.int32, cacheable=True)
c = allocate(shape=((DIM, DIM)), dtype=np.int32, cacheable=True)

a[:] = np.random.randint(0,100,size=(DIM, DIM))
b[:] = np.random.randint(0,100,size=(DIM, DIM))
c[:] = np.zeros((DIM, DIM)).astype('int')

print(a)
print(b)
print(c)

#### Set the address value from .../impl/ip/drivers/.../src/xmatmul_hw.h

In [ ]:
XMATMUL_CONTROL_ADDR_A_DATA = 0x10
XMATMUL_CONTROL_ADDR_B_DATA = 0x18
XMATMUL_CONTROL_ADDR_C_DATA = 0x20

#### Write data to input port

In [ ]:
a_ptr = a.physical_address
b_ptr = b.physical_address
c_ptr = c.physical_address

ip.write(0x00, 4)
fpga_state = ip.read(0x00)

if fpga_state == 4:
    ip.write(XMATMUL_CONTROL_ADDR_A_DATA, a_ptr)
    ip.write(XMATMUL_CONTROL_ADDR_B_DATA, b_ptr)
    ip.write(XMATMUL_CONTROL_ADDR_C_DATA, c_ptr)
else:
    print("Can't write values, must be in IDLE state")
    raise KeyboardInterrupt

#### Start multiplication

In [ ]:
ip.write(0x00, 1)
fpga_state = ip.read(0x00)

max_try = 1000000
while fpga_state != 6 and fpga_state != 4:
    fpga_state = ip.read(0x00)
    max_try = max_try - 1
    if max_try == 0:
        print("ERROR: Can't go ahead")
        ip.write(0x00, 4)
        raise KeyboardInterrupt
        
print(c)

#### Verify the output

In [ ]:
if (c == a*b).any():
    print("Correct!")
else:
    print("Wrong!")

#### Clean up

In [ ]:
del ol